In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import numpy as np
import json
import tweepy as tw
import nltk
import matplotlib.cm as cm
from nltk.corpus import stopwords
from nltk import bigrams
import re
import networkx as nx
import igraph
import warnings
import community as community_louvain
warnings.filterwarnings("ignore")

In [2]:
tweet_nonmis = []
x=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
filename='./tweets/coronavirus_smaller_aaa'
for i in range(len(x)):
    file=filename+x[i]+'.json'
    for line in open(file):
        tweet_nonmis.append(json.loads(line))
        

In [3]:
def load_reduced_tweet(tweet):
    if 'referenced_tweets' in tweet: 
        ref_tweets_type= tweet['referenced_tweets'][0]['type']
        ref_tweets_id= tweet['referenced_tweets'][0]['id']
    else:
        ref_tweets_type= None
        ref_tweets_id= None
    reduced_tweet = {
                        'author_id' : tweet['author_id'],
                        'id' : tweet['id'],
                        'referenced_tweets_id' : ref_tweets_id,
                      }
    return reduced_tweet

def load_tweets(tweets):
    reduced_tweets=[]
    for tweet in tweets:
        if 'author_id' in tweet:
            reduced_tweets.append(load_reduced_tweet(tweet))
        elif 'tweets' in tweet:
                for a_single_tweet in tweet["tweets"]:
                    reduced_tweets.append(load_reduced_tweet(a_single_tweet))
    return reduced_tweets




In [ ]:
reduced_tweet_nonmis = load_tweets(tweet_nonmis)
print(len(reduced_tweet_nonmis))

In [21]:
df_tweet_nonmis = pd.DataFrame(reduced_tweet_nonmis)
df_tweet_nonmis['ifmis']=0
df_tweet_nonmis.drop_duplicates(subset=['id'],inplace=True)
print(len(df_tweet_nonmis))
df_tweet_nonmis.to_csv("./dataset/df_tweet_nonmis.csv",index=False)


NameError: name 'reduced_tweet_nonmis' is not defined

In [3]:
df_tweet_nonmis = pd.read_csv("./dataset/df_tweet_nonmis.csv")
df_tweet_nonmis

,author_id,id,referenced_tweets_id,ifmis
0,1289288731430498304,1357411152884453377,NaN,0
1,3197617796,1357411152792342530,NaN,0
2,228793151,1357411152733605888,NaN,0
3,1275531383021146112,1357411152720928768,1.357351e+18,0
4,344269254,1357411152368648193,1.357410e+18,0
...,...,...,...,...
1696901,720739585,1357158157160747008,1.356974e+18,0
1696902,815382934974668801,1357158156976050177,1.357086e+18,0
1696903,777949323912937472,1357158156816707588,1.356434e+18,0
1696904,1223740046165053441,1357158156682592260,NaN,0


In [4]:
df_tweet_nonmis = df_tweet_nonmis.fillna(0)
df_tweet_nonmis['referenced_tweets_id']=df_tweet_nonmis['referenced_tweets_id'].astype(int)

In [5]:
df_tweet_nonmis

,author_id,id,referenced_tweets_id,ifmis
0,1289288731430498304,1357411152884453377,0,0
1,3197617796,1357411152792342530,0,0
2,228793151,1357411152733605888,0,0
3,1275531383021146112,1357411152720928768,1357351156826083328,0
4,344269254,1357411152368648193,1357410121274523648,0
...,...,...,...,...
1696901,720739585,1357158157160747008,1356974181360037888,0
1696902,815382934974668801,1357158156976050177,1357086106030911488,0
1696903,777949323912937472,1357158156816707588,1356434213352837120,0
1696904,1223740046165053441,1357158156682592260,0,0


In [13]:
tweet_mis = []
for line in open('./dataset/covid-misinfo-terms-small.json'):
    tweet_mis.append(json.loads(line))

In [14]:
reduced_tweet_mis = load_tweets(tweet_mis)
print(len(reduced_tweet_mis))
df_tweet_mis = pd.DataFrame(reduced_tweet_mis)
df_tweet_mis['ifmis']=1
df_tweet_mis.drop_duplicates(subset=['id'],inplace=True)

print(len(df_tweet_mis))
df_tweet_mis.to_csv("./dataset/df_tweet_mis.csv",index=False)

244449
213110


In [6]:
df_tweet_mis = pd.read_csv("./dataset/df_tweet_mis.csv")
df_tweet_mis

,author_id,id,referenced_tweets_id,ifmis
0,126237449,1366668782677069831,1.366377e+18,1
1,1245452098935730176,1366668731124838401,1.366110e+18,1
2,348795749,1366668486567731201,NaN,1
3,1075763979778187264,1366668198364479488,1.366514e+18,1
4,270416438,1366667937298472960,1.366657e+18,1
...,...,...,...,...
213105,3349241,1355667443784495107,1.355663e+18,1
213106,1338046991914291201,1355379200404819971,NaN,1
213107,138182116,1355257433992671235,NaN,1
213108,3308416363,1355638583688908817,1.355618e+18,1


In [7]:
df_tweet_mis = df_tweet_mis.fillna(0)
df_tweet_mis['referenced_tweets_id']=df_tweet_mis['referenced_tweets_id'].astype(int)

In [8]:
df_tweet_mis

,author_id,id,referenced_tweets_id,ifmis
0,126237449,1366668782677069831,1366377045694029824,1
1,1245452098935730176,1366668731124838401,1366109545727098880,1
2,348795749,1366668486567731201,0,1
3,1075763979778187264,1366668198364479488,1366514380557402112,1
4,270416438,1366667937298472960,1366657340234231808,1
...,...,...,...,...
213105,3349241,1355667443784495107,1355662729151827968,1
213106,1338046991914291201,1355379200404819971,0,1
213107,138182116,1355257433992671235,0,1
213108,3308416363,1355638583688908817,1355618128567402496,1


In [9]:
aid=df_tweet_mis.id.tolist()
bid=df_tweet_mis.author_id.tolist()
print(len(aid),len(bid))

213110 213110


In [10]:
cid=df_tweet_nonmis.id.tolist()
did=df_tweet_nonmis.author_id.tolist()
print(len(cid),len(did))

1696906 1696906


In [11]:
dic_id1={aid[i]:bid[i] for i in range(len(aid))}
print(len(dic_id1))

213110


In [12]:
dic_id2={cid[i]:did[i] for i in range(len(cid))}
print(len(dic_id2))

1696906


In [13]:
dic_id={}
dic_id.update(dic_id1)
dic_id.update(dic_id2)
len(dic_id)

1907184

In [14]:
dic_id

{1366668782677069831: 126237449,
 1366668731124838401: 1245452098935730176,
 1366668486567731201: 348795749,
 1366668198364479488: 1075763979778187264,
 1366667937298472960: 270416438,
 1366667935390060544: 1252904921416032256,
 1366667815118278656: 723575754722668544,
 1366667738052128773: 1326247197399150592,
 1366667608011907072: 1273194346834087936,
 1366667529712590851: 192835898,
 1366667361466605571: 1231420116996218881,
 1366667160500596743: 617838562,
 1366667048709877764: 1361967781206319107,
 1366667006645329920: 1358686679930920960,
 1366666981387214850: 1239022454862950401,
 1366666771961319429: 1153316743,
 1366666428888326146: 1259413557910921216,
 1366666373720604674: 1352609518732144640,
 1366666170250764289: 1157444775135666176,
 1366666001069330434: 570816464,
 1366665990726176768: 1361731033377439747,
 1366665868197912576: 868346664129581056,
 1366665703630254087: 1349346310126661635,
 1366665572063338496: 366625965,
 1366665479331471362: 1181691532618940416,
 13666

In [15]:
def find_sourse_author(x):
    if x == 0:
        return None
    else:
        sid=dic_id.setdefault(x, None)
        return sid

df_tweet_mis["Source_author_id"] = df_tweet_mis['referenced_tweets_id'].apply(find_sourse_author)
df_tweet_nonmis["Source_author_id"] = df_tweet_nonmis['referenced_tweets_id'].apply(find_sourse_author)

In [16]:
print(len(df_tweet_mis),len(df_tweet_nonmis))

213110 1696906


In [19]:
df_tweet_nonmis.dropna(subset=['Source_author_id'],inplace=True)
df_tweet_mis.dropna(subset=['Source_author_id'],inplace=True)
print(len(df_tweet_mis),len(df_tweet_nonmis))

33113 381850


In [20]:
data = pd.concat([df_tweet_mis, df_tweet_nonmis], ignore_index=True)
data

,author_id,id,referenced_tweets_id,ifmis,Source_author_id
0,1245452098935730176,1366668731124838401,1366109545727098880,1,1.077215e+09
1,1326247197399150592,1366667738052128773,1366494764556812288,1,8.204525e+17
2,1361967781206319107,1366667048709877764,1366650505179455488,1,2.208796e+08
3,1144482234860052480,1366665373806956549,1366658733711974400,1,1.502464e+08
4,769301900378374144,1366665173176573955,1366044315051569152,1,1.295647e+18
...,...,...,...,...,...
414958,78346318,1357158158888804353,1357130453615079424,0,1.472711e+08
414959,1868151614,1357158158851055616,1356955013441028096,0,8.606545e+17
414960,912815659,1357158158460928000,1357141885387759616,0,1.011971e+18
414961,187535831,1357158157919784960,1357133486201659392,0,1.935743e+09


In [21]:
data['Source_author_id']=data['Source_author_id'].astype(int)

In [23]:
nonmis_network_data= df_tweet_nonmis[['author_id', 'Source_author_id']]
nonmis_network_data['weight']=0
nonmis_network_data=nonmis_network_data.groupby(['author_id', 'Source_author_id'], as_index=False).count()
nonmis_network_data['ifmis']=0
mis_network_data= df_tweet_mis[['author_id', 'Source_author_id']]
mis_network_data['weight']=0
mis_network_data=mis_network_data.groupby(['author_id', 'Source_author_id'], as_index=False).count()
mis_network_data['ifmis']=1
network_data=pd.concat([nonmis_network_data, mis_network_data], ignore_index=True)
print(len(mis_network_data),len(nonmis_network_data))

29887 361711


In [24]:
G=nx.from_pandas_edgelist(network_data, 'author_id', 'Source_author_id', ["weight","ifmis"])

In [26]:
G_mis = nx.from_pandas_edgelist(mis_network_data, 'author_id', 'Source_author_id', ["weight","ifmis"], create_using = nx.DiGraph())
G_nonmis = nx.from_pandas_edgelist(nonmis_network_data, 'author_id', 'Source_author_id', ["weight","ifmis"], create_using = nx.DiGraph())

In [28]:
G_componrnts=nx.strongly_connected_components(G_mis)
largest_components=max(G_componrnts,key=len)
subgraph_nodes=list(largest_components)
G_subgraph=G_mis.subgraph(subgraph_nodes)
H=G_subgraph.to_undirected()
print("Number of Gmis nodes:",H.number_of_nodes())
print("Number of Gmis edges:",H.size())

Number of Gmis nodes: 14
Number of Gmis edges: 27


In [29]:
G_componrnts=nx.strongly_connected_components(G_nonmis)
largest_components=max(G_componrnts,key=len)
subgraph_nodes=list(largest_components)
G_subgraph=G_nonmis.subgraph(subgraph_nodes)
H=G_subgraph.to_undirected()
print("Number of Gnonmis nodes:",H.number_of_nodes())
print("Number of Gnonmis edges:",H.size())

Number of Gnonmis nodes: 30
Number of Gnonmis edges: 76


In [25]:
G_mis = nx.from_pandas_edgelist(mis_network_data, 'author_id', 'Source_author_id', ["weight","ifmis"])
G_nonmis = nx.from_pandas_edgelist(nonmis_network_data, 'author_id', 'Source_author_id', ["weight","ifmis"])

In [35]:
Gmis_componrnts=nx.connected_components(G_mis)
largest_components=max(Gmis_componrnts,key=len)
subgraph_nodes=list(largest_components)
Gmis_subgraph=G_mis.subgraph(subgraph_nodes)

print("Number of g nodes:",Gmis_subgraph.number_of_nodes())
print("Number of g edges:",Gmis_subgraph.size())

Number of g nodes: 20246
Number of g edges: 24142


In [37]:
Gnmis_componrnts=nx.connected_components(G_nonmis)
largest_components=max(Gnmis_componrnts,key=len)
subgraph_nodes=list(largest_components)
Gnmis_subgraph=G_nonmis.subgraph(subgraph_nodes)
print("Number of g nodes:",Gnmis_subgraph.number_of_nodes())
print("Number of g edges:",Gnmis_subgraph.size())

Number of g nodes: 245138
Number of g edges: 301579


In [41]:
df_Gnmis_subgraph = nx.to_pandas_edgelist(Gnmis_subgraph)
df_Gmis_subgraph = nx.to_pandas_edgelist(Gmis_subgraph)


In [42]:
df_Gnmis_subgraph

,source,target,weight,ifmis
0,1.870000e+02,2.927380e+09,1,0
1,2.927380e+09,3.206000e+03,1,0
2,2.927380e+09,5.664472e+06,1,0
3,2.927380e+09,6.296522e+06,1,0
4,2.927380e+09,6.576662e+06,1,0
...,...,...,...,...
301574,1.357265e+18,1.357265e+18,2,0
301575,1.357281e+18,1.356478e+18,1,0
301576,1.357372e+18,3.577780e+08,1,0
301577,1.357387e+18,2.923341e+08,1,0


In [43]:
df_Gmis_subgraph

,source,target,weight,ifmis
0,2.692000e+03,1.163619e+09,1,1
1,1.163619e+09,3.384441e+06,1,1
2,1.163619e+09,4.937061e+06,1,1
3,1.163619e+09,7.848802e+06,1,1
4,1.163619e+09,9.412542e+06,1,1
...,...,...,...,...
24137,1.365013e+18,8.612725e+17,1,1
24138,1.365013e+18,1.235346e+18,1,1
24139,1.365013e+18,1.242887e+18,3,1
24140,1.365013e+18,1.335035e+18,1,1


In [47]:
df_Gmis_subgraph['source']=df_Gmis_subgraph['source'].astype(int)
df_Gmis_subgraph['target']=df_Gmis_subgraph['target'].astype(int)
df_Gnmis_subgraph['source']=df_Gnmis_subgraph['source'].astype(int)
df_Gnmis_subgraph['target']=df_Gnmis_subgraph['target'].astype(int)

In [50]:
data_subgraph = pd.concat([df_Gmis_subgraph, df_Gnmis_subgraph], ignore_index=True)
data_subgraph

,source,target,weight,ifmis
0,2692,1163618839,1,1
1,1163618839,3384441,1,1
2,1163618839,4937061,1,1
3,1163618839,7848802,1,1
4,1163618839,9412542,1,1
...,...,...,...,...
325716,1357264512060112896,1357264512060112896,2,0
325717,1357281486869655552,1356478228899061760,1,0
325718,1357371859180822528,357777980,1,0
325719,1357387387148582912,292334056,1,0


In [51]:
def apply_id(x):
    index=dic_id[x]
    return index
id1=data_subgraph.source.unique().tolist()
id2=data_subgraph.target.unique().tolist()
id3=id1+id2
id4 = list(set(id3)) 
dic_id = {id4[i]:i for i in range(len(id4))}
data_subgraph["source"] = data_subgraph["source"].apply(apply_id)
data_subgraph["target"]  = data_subgraph["target"].apply(apply_id)
data_subgraph.sort_values("source",inplace=True)
data_subgraph['ifmis'].value_counts()

0    301579
1     24142
Name: ifmis, dtype: int64

In [52]:
df_node1 = data_subgraph.groupby(['source'])['ifmis'].max()
df_node2 = data_subgraph.groupby(['target'])['ifmis'].max()
df_node1 = pd.DataFrame(df_node1)
df_node1 = df_node1.reset_index()
new_col = ['node', 'lable']
df_node1.columns = new_col
df_node2 = pd.DataFrame(df_node2)
df_node2 = df_node2.reset_index()
df_node2.columns = new_col
df_node = pd.merge(df_node1,df_node2,on='node',how='outer') 
df_node['lable_x'].fillna(0, inplace=True)
df_node['lable_y'].fillna(0, inplace=True)
df_node['lable']=df_node['lable_x']+df_node['lable_y']
df_node = df_node[['node','lable']]
df_node.sort_values("node",inplace=True)
df_node

,node,lable
33221,0,0.0
33222,1,0.0
33223,2,0.0
33224,3,0.0
33225,4,0.0
...,...,...
258847,258846,0.0
258848,258847,0.0
258849,258848,0.0
258850,258849,0.0


In [53]:
df_node['lable'].value_counts()

0.0    238887
1.0     16876
2.0      3088
Name: lable, dtype: int64

In [54]:
def setlable(x):
    if x>=1:
        return 1
    else:
        return 0

df_node["lable1"] = df_node["lable"] .apply(setlable)
df_node

,node,lable,lable1
33221,0,0.0,0
33222,1,0.0,0
33223,2,0.0,0
33224,3,0.0,0
33225,4,0.0,0
...,...,...,...
258847,258846,0.0,0
258848,258847,0.0,0
258849,258848,0.0,0
258850,258849,0.0,0


In [56]:
df_node['lable1'].value_counts()

0    238887
1     19964
Name: lable1, dtype: int64

In [57]:
node=df_node.node.tolist()
node_lable=df_node.lable1.tolist()
dic_node = {node[i]:node_lable[i] for i in range(len(node))}

In [1]:
G = igraph.Graph.DataFrame(data_subgraph, directed=False)

NameError: name 'igraph' is not defined

In [58]:
g = nx.from_pandas_edgelist(data_subgraph, 'source', 'target', ['weight','ifmis'])

In [59]:
nx.set_node_attributes(g, dic_node, "ismis")

In [60]:
print("Number of g nodes:",g.number_of_nodes())
print("Number of g edges:",g.size())

Number of g nodes: 258851
Number of g edges: 325100


In [61]:
def drawgraph(g,menberships):# draw the graph
    pos = nx.spring_layout(g)
    # color the nodes according to their partition
    cmap = cm.get_cmap('viridis', max(menberships.values()) + 1)
    nx.draw_networkx_nodes(g, pos, menberships.keys(), node_size=10,
                           cmap=cmap, node_color=list(menberships.values()),
                          with_labels=False)
    nx.draw_networkx_edges(g, pos, alpha=0.5)
    plt.show()


In [ ]:
drawgraph(g,dic_node)